# S3DIS inference
This notebook lets you run inference on the S3DIS Test Fold, from a pretrained model's checkpoint on your machine. The output will be a confusion matrix which you can save locally on your machine. For this, you must have done the following:
- downloaded and preprocessed the S3DIS dataset (if you haven't, instantiating the dataset will launch it for you though)
- trained a model (which produced a checkpoint directory) or downloaded our pretrained weights

Note this notebook should work for multimodal (2D+3D) and unimodal (3D only) experiments.

In [ ]:
# Select you GPU
I_GPU = 0

In [ ]:
# Uncomment to use autoreload
%load_ext autoreload
%autoreload 2

import os
import sys
import numpy as np
import torch
import glob
from omegaconf import OmegaConf
import warnings
warnings.filterwarnings('ignore')

torch.cuda.set_device(I_GPU)
DIR = os.path.dirname(os.getcwd())
ROOT = os.path.join(DIR, "..")
sys.path.insert(0, ROOT)
sys.path.insert(0, DIR)

from torch_points3d.utils.config import hydra_read
from torch_points3d.trainer import Trainer

In [ ]:
# Set your parameters
DATA_ROOT = '/path/to/your/dataset/root/directory'
checkpoint_dir = '/directory/containing/your/checkpoint/files'        # each fold's checkpoint should: follow checkpoint_dir/area_i/model_name.pt
result_dir = '/directory/where/to/save/inference/metrics'
model_name = 'Res16UNet34-L4-early'                                   # adapt if you use another model in your checkpoint
eval_sample_res = 1                                                   # controls the sphere sampling grid resolution. The lower, the more val/test spheres and the higher their overlap
n_votes = 1                                                           # number of inferences per spherical sample. For multi-inference voting with inference-time augmentation
batch_size = 8                                                        # increase if your device allows it
full_res = True                                                       # predictions will be made on the raw point cloud, at full resolution
num_workers = 4                                                       # increase if your machine allows it
exp_name = None                                                       # you may give a name to the experiment
exp_name = f'{model_name}_votes-{n_votes}_sample_res-{eval_sample_res}' if exp_name is None else exp_name

In [ ]:
print(f'Inference on S3DIS exp={exp_name}')

for i_area in range(6):
    print(f'\nInference on Area {i_area + 1}')
    
    # These are the arguments passed to Hydra. You could run the same thing 
    # from CLI using `eval.py` with the command `python eval.py [...]`
    overrides = [
        f'model_name={model_name}',
        f"checkpoint_dir={os.path.join(checkpoint_dir, f'area_{i + 1}')}",
        f'voting_runs={n_votes}',
        f'tracker_options.full_res={full_res}',
        'precompute_multi_scale=False',
        f'num_workers={num_workers}',
        f'batch_size={batch_size}',
        f'cuda={I_GPU}',
        'weight_name=latest',
        f'+data.dataroot={DATA_ROOT}',
        f'+data.eval_sample_res={eval_sample_res}',
    ]

    # Parse the arguments with Hydra and OmegaConf
    cfg = hydra_read(overrides, config_name='eval')
    OmegaConf.set_struct(cfg, False)

    # Create the Trainer instance from your checkpoint
    trainer = Trainer(cfg)

    # Update the test transforms to match train transforms for test-time 
    # augmentation
    trainer._dataset.test_dataset[0].transform = trainer._dataset.train_dataset.transform
    if trainer._model.is_multimodal:
        trainer._dataset.test_dataset[0].transform_image = trainer._dataset.train_dataset.transform_image
        trainer._dataset.test_dataset[0].transform_image.transforms[4].use_coverage = False
        trainer._dataset.test_dataset[0].transform_image.transforms[4].credit = int(2097152 * 2)
        trainer._dataset.test_dataset[0].transform_image.transforms[5].sigma = 0.02
        trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.saturation = [0.8, 1.2]
        trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.brightness = [0.8, 1.2]
        trainer._dataset.test_dataset[0].transform_image.transforms[6].transform.contrast = [0.8, 1.2]

    # Run inference
    trainer.eval(stage_name='test')
    cm = trainer._tracker._full_confusion_matrix
    torch.save(cm, f'{result_dir}/{exp_name}_area-{i_area + 1}.pt')
    print(f'  mIoU={cm.get_average_intersection_union() * 100:0.2f}')
    print(f'  OA={cm.get_overall_accuracy() * 100:0.2f}')

Confusion matrices will be saved in your `result_dir`.

Now, if we want to compute the 6-Fold performance, we need to accumulate the confusion matrices from all the folds into a single one.

In [ ]:
cm_6fold = None

for i_area in range(6):
    
    # Load the confusion matrix previously computed
    path = f'{result_dir}/{exp_name}_area-{i_area + 1}.pt'
    cm_fold = torch.load(path)
    
    # Simple case for the first fold
    if i_area == 0:
        cm_6fold = cm_fold
        continue
    
    # Accumulate the CMs from all folds
    cm_6fold.confusion_matrix += cm_fold.confusion_matrix

    # Save the 6-fold CM
torch.save(cm_6fold, f'{result_dir}/{exp_name}_6-fold.pt')
    
print(f'6-Fold inference')
print(f'  mIoU={cm_6fold.get_average_intersection_union() * 100:0.2f}')
print(f'  OA={cm_6fold.get_overall_accuracy() * 100:0.2f}')

Please refer to `torch_points3d/metrics/confusion_matrix` if you need to compute more metrics from the confusion matrices.